<a href="https://colab.research.google.com/github/yoheikikuta/US-patent-analysis/blob/master/colab/feature_extract_W2V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature extraction from w2v model



In [0]:
from google.colab import auth
auth.authenticate_user()

## Data preparation

In [0]:
DATA_DIR = "./"

In [3]:
!gsutil cp gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/grants_for_3000+3000.df.gz {DATA_DIR}
!gsutil cp gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/testset_app_3000.df.gz {DATA_DIR}
!gsutil cp gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/training_app_3000.df.gz {DATA_DIR}

Copying gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/grants_for_3000+3000.df.gz...
- [1 files][129.4 MiB/129.4 MiB]                                                
Operation completed over 1 objects/129.4 MiB.                                    
Copying gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/testset_app_3000.df.gz...
\ [1 files][ 45.5 MiB/ 45.5 MiB]                                                
Operation completed over 1 objects/45.5 MiB.                                     
Copying gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-xml/training_app_3000.df.gz...
\ [1 files][ 45.0 MiB/ 45.0 MiB]                                                
Operation completed over 1 objects/45.0 MiB.                                     


In [0]:
import gzip
import pickle
import datetime
import json
import os
import pprint
import random
import string
import sys
import pandas as pd

In [0]:
grants = pd.read_pickle(f"{DATA_DIR}grants_for_3000+3000.df.gz")
test_app = pd.read_pickle(f"{DATA_DIR}testset_app_3000.df.gz")
train_app = pd.read_pickle(f"{DATA_DIR}training_app_3000.df.gz")

In [0]:
import re


CLAIM_PAT = re.compile(r'<claims[^>]*>(.*)</claims>',re.MULTILINE|re.DOTALL)
TAG_PAT = re.compile(r"<.*?>")
LB_PAT = re.compile(r'[\t\n\r\f\v][" "]*')
CANCELED_PAT = re.compile(r'[0-9]+.*\. \(canceled\)[" "]')
NUM_PAT = re.compile(r'[" "]?[0-9]+[" "]?\.[" "]?')


def whole_xml_to_claim_xml(whole):
    mat = CLAIM_PAT.search(whole)
    return mat.group(1)


def whole_xml_to_claim(whole):
    return TAG_PAT.sub(' ', whole_xml_to_claim_xml(whole))


def remove_linebreak_from_claim(claim):
    return LB_PAT.sub('', claim)


def remove_canceled_claim(claim):
    return CANCELED_PAT.sub('', claim)


def remove_claim_numbers(claim):
    return NUM_PAT.sub('', claim)  

In [7]:
%%time

train_app["claim_app"] = train_app["xml"].map(whole_xml_to_claim).map(remove_canceled_claim).map(remove_claim_numbers).map(remove_linebreak_from_claim)
train_app = train_app.drop("xml", axis=1)
train_app.head()

test_app["claim_app"] = test_app["xml"].map(whole_xml_to_claim).map(remove_canceled_claim).map(remove_claim_numbers).map(remove_linebreak_from_claim)
test_app = test_app.drop("xml", axis=1)
test_app.head()

grants["claim_cited_grant"] = grants["xml"].map(whole_xml_to_claim).map(remove_canceled_claim).map(remove_claim_numbers).map(remove_linebreak_from_claim)
grants = grants.drop("xml", axis=1)
grants.head()

CPU times: user 9.01 s, sys: 185 ms, total: 9.2 s
Wall time: 9.2 s


In [8]:
train_app.head(3)

,app_id,claim_app
0,12130785,A system for differentiating noise from an arr...
1,12652424,A method of allocating resources in a data war...
2,12214532,A controlling method of a media processing app...


## Feature extraction: Word2Vec

In [0]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from scipy import spatial

import multiprocessing
CPUNUM = multiprocessing.cpu_count()

In [0]:
### Use (train + grants) for w2v training


all_training_list = train_app['claim_app'].map(lambda x:x.split()).tolist() + grants['claim_cited_grant'].map(lambda x:x.split()).tolist()

In [0]:
len(all_training_list)

9440

In [0]:
all_training_list[0][:5]

['A', 'system', 'for', 'differentiating', 'noise']

In [0]:
%%time

w2v = Word2Vec(
    all_training_list,
    size=100, window=5, min_count=5, workers=CPUNUM, iter=10, hs=1, seed=23
)

CPU times: user 7min 29s, sys: 1.61 s, total: 7min 31s
Wall time: 3min 50s


In [0]:
w2v.most_similar('generator')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('generator,', 0.6032677888870239),
 ('generation', 0.5030529499053955),
 ('oscillator', 0.5020977258682251),
 ('detector', 0.496626079082489),
 ('controller', 0.4901273846626282),
 ('regulator', 0.4858677089214325),
 ('driver', 0.48575833439826965),
 ('source', 0.48183900117874146),
 ('sensor', 0.481015145778656),
 ('motor', 0.46535730361938477)]

In [0]:
GS_BASE = "gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-extracted-feature/W2V"

def dump_and_send(fname, obj):
  with gzip.open(fname, 'w') as f:
     pickle.dump(obj, f)
  !gsutil cp {fname} {GS_BASE}/{fname}
  print(f"send to {GS_BASE}/{fname}")

In [0]:
def text_to_vec(text):
    words = text.split()
    filtered_words = []
    
    for word in words:
        if word in w2v.wv.vocab:
            filtered_words.append(word)
    vec = np.average(w2v[filtered_words], axis=0)
    return vec

In [0]:
%%time

train_features = np.array([text_to_vec(claim) for claim in train_app['claim_app']])
dump_and_send("w2v_100_feature_train_app_3000.pkl.gz", train_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Copying file://w2v_100_feature_train_app_3000.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/1.1 MiB.                                      
send to gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-extracted-feature/W2V/w2v_100_feature_train_app_3000.pkl.gz
CPU times: user 9.79 s, sys: 104 ms, total: 9.89 s
Wall time: 14.5 s


In [0]:
train_features.shape

(3000, 100)

In [0]:
train_features[0][:5]

array([-0.30962306, -0.01635869,  0.04177235, -0.27590832,  0.02144193],
      dtype=float32)

In [0]:
train_features[0][-5:]

array([ 0.40655544, -0.37481394, -0.82976675,  0.00378964,  0.11228371],
      dtype=float32)

In [0]:
%%time

test_features = np.array([text_to_vec(claim) for claim in test_app['claim_app']])
dump_and_send("w2v_100_feature_test_app_3000.pkl.gz", test_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Copying file://w2v_100_feature_test_app_3000.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/1.1 MiB.                                      
send to gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-extracted-feature/W2V/w2v_100_feature_test_app_3000.pkl.gz
CPU times: user 9.98 s, sys: 98.9 ms, total: 10.1 s
Wall time: 13.7 s


In [0]:
%%time

grants_features = np.array([text_to_vec(claim) for claim in grants['claim_cited_grant']])
dump_and_send("w2v_100_feature_grants_3000_3000.pkl.gz", grants_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Copying file://w2v_100_feature_grants_3000_3000.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/2.3 MiB.                                      
send to gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-extracted-feature/W2V/w2v_100_feature_grants_3000_3000.pkl.gz
CPU times: user 28.2 s, sys: 181 ms, total: 28.3 s
Wall time: 32.3 s


## Feature extraction: fasttext

In [0]:
import pandas as pd
import numpy as np
from scipy import spatial

import multiprocessing
CPUNUM = multiprocessing.cpu_count()

In [10]:
import gensim.downloader as api

model = api.load("fasttext-wiki-news-subwords-300")  # download the model and return as object ready for use

[==================================================] 100.0% 958.5/958.4MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
model.most_similar("generator")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('generators', 0.8244898319244385),
 ('cogenerator', 0.7042113542556763),
 ('Generator', 0.6945025324821472),
 ('motor-generator', 0.6781628131866455),
 ('alternator', 0.6766893863677979),
 ('turbine-generator', 0.6669933795928955),
 ('engine-generator', 0.6461780071258545),
 ('turbogenerator', 0.6457508206367493),
 ('turbo-generator', 0.6451094150543213),
 ('regenerator', 0.6378977298736572)]

In [16]:
list(model.vocab.keys())[:10], list(model.vocab.keys())[-5:]

([',', 'the', '.', 'and', 'of', 'to', 'in', 'a', '"', ':'],
 ['whitespotted', 'sacoglossan', 'Iseya', 'Bayyah', 'Vilaya'])

In [20]:
model['generator'].shape

(300,)

In [0]:
GS_BASE = "gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-extracted-feature/fastText"

def dump_and_send(fname, obj):
  with gzip.open(fname, 'w') as f:
     pickle.dump(obj, f)
  !gsutil cp {fname} {GS_BASE}/{fname}
  print(f"send to {GS_BASE}/{fname}")

In [0]:
def text_to_vec(text):
    words = text.split()
    filtered_words = []
    
    for word in words:
        if word in model.vocab:
            filtered_words.append(word)
    vec = np.average(model[filtered_words], axis=0)
    return vec

In [21]:
%%time

train_features = np.array([text_to_vec(claim) for claim in train_app['claim_app']])
dump_and_send("fasttext_300_feature_train_app_3000.pkl.gz", train_features)

Copying file://fasttext_300_feature_train_app_3000.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/3.2 MiB.                                      
send to gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-extracted-feature/fastText/fasttext_300_feature_train_app_3000.pkl.gz
CPU times: user 9.49 s, sys: 158 ms, total: 9.65 s
Wall time: 15.5 s


In [22]:
train_features.shape

(3000, 300)

In [23]:
train_features[0][:5]

array([ 0.00127815, -0.00225469,  0.02024586,  0.01990329, -0.02750648],
      dtype=float32)

In [24]:
%%time

test_features = np.array([text_to_vec(claim) for claim in test_app['claim_app']])
dump_and_send("fasttext_300_feature_test_app_3000.pkl.gz", test_features)

Copying file://fasttext_300_feature_test_app_3000.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/3.2 MiB.                                      
send to gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-extracted-feature/fastText/fasttext_300_feature_test_app_3000.pkl.gz
CPU times: user 9.7 s, sys: 220 ms, total: 9.92 s
Wall time: 14.7 s


In [25]:
%%time

grants_features = np.array([text_to_vec(claim) for claim in grants['claim_cited_grant']])
dump_and_send("fasttext_300_feature_grants_3000_3000.pkl.gz", grants_features)

Copying file://fasttext_300_feature_grants_3000_3000.pkl.gz [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/6.9 MiB.                                      
send to gs://yohei-kikuta/mlstudy-phys/patent-analysis/3000-extracted-feature/fastText/fasttext_300_feature_grants_3000_3000.pkl.gz
CPU times: user 28.6 s, sys: 382 ms, total: 29 s
Wall time: 34 s
